In [24]:
import os
import sys
import timeit
import numpy as np
import theano
import theano.tensor as T
import lasagne
from theano.tensor.signal import pool
from theano.tensor.nnet import conv2d
from logistic_sgd import load_data
from lasagne.layers import get_output, InputLayer, DenseLayer, Upscale2DLayer, ReshapeLayer
import gzip
import pickle

In [25]:
batch_size = 1000
f = gzip.open('/home/rui/Downloads/mnist.pkl.gz', 'rb')
try:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
except:
    train_set, valid_set, test_set = pickle.load(f)
f.close()
X_train, y_train = train_set
y_train = np.asarray(y_train, dtype = np.int32)
X_test, y_test = test_set
y_test = np.asarray(y_test, dtype = np.int32)

In [26]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]



In [57]:
class LogisticRegression(object):
    def __init__(self, X, y, num_units=10):
        self.X = X
        self.y = y
        self.input_var = T.matrix('inputs')
        self.target_var = T.ivector('targets')
        self.l_in = InputLayer(shape=(None, X.shape[1]), input_var=self.input_var)
        self.l_out = DenseLayer(self.l_in, num_units=num_units, nonlinearity=lasagne.nonlinearities.softmax)
        self.prediction = lasagne.layers.get_output(self.l_out)
        self.loss = lasagne.objectives.categorical_crossentropy(self.prediction, self.target_var)
        self.loss = self.loss.mean()
        self.params = lasagne.layers.get_all_params(self.l_out, trainable=True)
        self.updates = lasagne.updates.adadelta(self.loss, self.params, learning_rate=1)
        
        self.test_prediction = lasagne.layers.get_output(self.l_out, deterministic=True)
        self.test_loss = lasagne.objectives.categorical_crossentropy(self.test_prediction,
                                                            self.target_var)
        self.test_loss = self.test_loss.mean()
        self.test_acc = T.mean(T.eq(T.argmax(self.test_prediction, axis=1), self.target_var),
                      dtype=theano.config.floatX)
        self.train_fn = theano.function([self.input_var, self.target_var],
                                        self.loss, updates=self.updates)
        self.acc_fn = theano.function([self.input_var, self.target_var], 
                                      self.test_acc)
        self.train()
        
    def train(self, num_epochs=50, batch_size=1000):
        for epoch in range(num_epochs):
            train_err = 0
            train_batches = 0
            for batch in iterate_minibatches(self.X, self.y, batch_size, shuffle=False):
                inputs, targets = batch
                train_err += self.train_fn(inputs, targets)
                train_batches += 1

    def eval_acc(self, X_test, y_test):
        test_acc = 0
        test_batches = 0
        for batch in iterate_minibatches(X_test, y_test, 1000, shuffle=False):
            inputs, targets = batch
            test_acc += self.acc_fn(inputs, targets)
            test_batches += 1
        self.test_acc = test_acc/test_batches
        print('overall acc is: {}'.format(self.test_acc))

In [60]:
lr = LogisticRegression(X_train, y_train)

# Logistic regression on original data

In [61]:
lr.eval_acc(X_test, y_testfirst layer)

overall acc is: 0.9243000090122223


In [ ]:
acc_list = []
acc_list.append(lr.test_acc)

In [70]:
input_var = T.matrix('inputs')
target_var = T.ivector('target')
l_in = InputLayer(shape=(None, 784), input_var=input_var)
pre_layer = l_in
hidden_layer_list = []
for i in range(20):
    h_layer = DenseLayer(pre_layer, num_units=512)
    hidden_layer_list.append(h_layer)
    pre_layer = h_layer
output_list = lasagne.layers.get_output(hidden_layer_list)
output_fn = theano.function([input_var], output_list)

In [71]:
feature_list = output_fn(X_train)

In [75]:
for x in feature_list:
    print(x.shape)

(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)
(50000, 512)


In [73]:
test_feature_list = output_fn(X_test)

In [76]:
for i in range(20):
    X = feature_list[i]
    lr = LogisticRegression(X, y_train)
    lr.eval_acc(test_feature_list[i], y_test)
    acc_list.append(lr.test_acc)

overall acc is: 0.9383000016212464
overall acc is: 0.9223999977111816
overall acc is: 0.8936000049114228
overall acc is: 0.8646999955177307
overall acc is: 0.8221999943256378
overall acc is: 0.7636000037193298
overall acc is: 0.6999000012874603
overall acc is: 0.6241000056266784
overall acc is: 0.5231000036001205
overall acc is: 0.424700003862381
overall acc is: 0.33680000007152555
overall acc is: 0.25219999700784684
overall acc is: 0.18709999918937684
overall acc is: 0.14549999982118605
overall acc is: 0.1169000007212162
overall acc is: 0.1135000005364418
overall acc is: 0.1135000005364418
overall acc is: 0.1135000005364418
overall acc is: 0.1135000005364418
overall acc is: 0.1135000005364418
